# RNN
## 기본 구조
* 시퀀스 데이터(순서가 있는 데이터) 처리하기 위해 고안된 구조
* 과거 정보를 기억해서 현재 예측에 반영함

## 구성요소
* 입력-은닉 가중치 (W_xh): 입력에서 은닉 상태로의 연결
* 은닉-은닉 가중치 (W_hh): 이전 은닉 상태에서 현재 은닉 상태로의 연결
* 은닉-출력 가중치 (W_hy): 은닉 상태에서 출력으로의 연결
* 편향 (b_h, b_y): 각 계층의 바이어스
* 은닉 상태 (h): 시간에 따라 업데이트되는 내부 상태

## 학습 방식
* 입력 시퀀스를 통과시켜 마지막 h를 구함
* 예측값 y_pred 계산
* 정답값 y_train과 비교하여 MSE Loss 계산
* loss.backward()를 호출하여 파라미터별 gradient 자동 계산
* optimizer.step()으로 파라미터 업데이트

## 요약 흐름도
window_size를 10으로 놓았을 때

입력 시퀀스 (x): [x0, x1, ..., x9]   →  출력 (y): x10
                    ↓
     h0 → h1 → ... → h9 → y_pred
> 각 x_t 입력마다 은닉 상태 h가 갱신되고, 마지막 시점의 h를 통해 y_pred (예측값)를 만듦

## 슬라이딩 윈도우
> 연속된 고정 길이 구간(window)을 시간축을 따라 이동시키며 시퀀스를 생성하는 방식
* RNN은 일정 길이의 시퀀스를 입력으로 받아야 하므로, df_train의 각 시계열에서 (window_size + 1) 만큼 잘라서 학습 샘플을 생성

In [6]:
import torch
import pandas as pd

# 데이터 로딩
df_train = pd.read_csv("hw2_dataset2_train.csv")
df_test = pd.read_csv("hw2_dataset2_test.csv")

# 하이퍼파라미터
window_size = 10
input_size = 1
hidden_size = 8
output_size = 1
learning_rate = 0.001
epochs = 2000

# Train 데이터 슬라이딩 윈도우 시퀀스 생성
train_sequences = []
for row in df_train.values:
    for i in range(len(row) - window_size):
        window = row[i:i+window_size+1]  # window + next step
        train_sequences.append(window)

train_sequences = torch.tensor(train_sequences, dtype=torch.float32)  # (N, window+1)
x_train = train_sequences[:, :-1].unsqueeze(-1)  # (N, window, 1)
y_train = train_sequences[:, -1].unsqueeze(-1)   # (N, 1)

# RNN 파라미터 정의
torch.manual_seed(0)
W_xh = torch.nn.Parameter(torch.randn(input_size, hidden_size))
W_hh = torch.nn.Parameter(torch.randn(hidden_size, hidden_size))
b_h = torch.nn.Parameter(torch.randn(hidden_size))
W_hy = torch.nn.Parameter(torch.randn(hidden_size, output_size))
b_y = torch.nn.Parameter(torch.randn(output_size))
params = [W_xh, W_hh, b_h, W_hy, b_y]

optimizer = torch.optim.Adam(params, lr=learning_rate)
loss_fn = torch.nn.MSELoss()

for epoch in range(epochs):
    h = torch.zeros(x_train.size(0), hidden_size)
    for t in range(window_size):
        x_t = x_train[:, t, :]
        h = torch.tanh(x_t @ W_xh + h @ W_hh + b_h)
    y_pred = h @ W_hy + b_y
    loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

# 이제 df_test 예측 수행
df_test_filled = df_test.copy()

# 컬럼 이름 추출
cols = list(df_test.columns)
known_cols = cols[:151]  # t=0.0 ~ t=15.0
missing_cols = cols[151:]  # t=15.1 ~

# 각 샘플에 대해 시간 순으로 다음 시점 예측
for i in range(len(df_test)):
    series = df_test.loc[i, known_cols].tolist()
    for c in missing_cols:
        x_seq = torch.tensor(series[-window_size:], dtype=torch.float32).view(1, window_size, 1)
        h = torch.zeros(1, hidden_size)
        for t in range(window_size):
            x_t = x_seq[:, t, :]
            h = torch.tanh(x_t @ W_xh + h @ W_hh + b_h)
        y_next = (h @ W_hy + b_y).item()
        df_test_filled.at[i, c] = y_next
        series.append(y_next)  # 예측값을 다음 입력으로 사용

# 결과 저장
name = "이유정"
df_test_filled.to_csv(f"hw2_dataset2_test_{name}_RNN.csv", index=False)

Epoch 0 | Loss: 19.479568
Epoch 500 | Loss: 0.033879
Epoch 1000 | Loss: 0.022749
Epoch 1500 | Loss: 0.015512


# RNN이 이렇게 느린 이유
* 순차적으로 처리해야해서 병렬 처리가 어렵고 한 시점씩 반복문으로 계산해야 함
* RNN의 역전파는 일반 네트워크보다 복잡함
* 시간 축으로 펼친 그래프에서 모든 시점의 gradient를 연쇄적으로 계산
* 시퀀스 길이가 길수록 계산량 기하급수적으로 증가
* 슬라이딩 윈도우를 사용하면 데이터 row*윈도우 크기 개수만큼의 샘플을 학습해야 하는데, 학습 루프마다 forward/backward 해야 하니 느려짐

# LSTM이랑 차이
* LSTM은 기존 RNN을 발전시킨 개선 모델
* 기존 RNN은 시퀀스가 길어질수록 역전파 시 gradient가 점점 0에 가까워져서 초기 정보가 학습되지 않아 Vanishing Gradient 문제가 발생함
* 기존 RNN은 장기 의존성 학습 실패하는 문제도 발생함
* LSTM은 내부에 cell state라는 긴 메모리를 두고 입력 게이트 / 삭제 게이트 / 출력 게이트로 정보를 선택적으로 기억하고 잊음
* 중요한 정보는 오래 보관, 불필요한 정보는 빠르게 삭제하고, 긴 시퀀스도 안정적으로 학습 가능하게 됨
> RNN: 사람이 매 순간 모든 걸 기억하려고 애쓰는 상태 → 금방 피로해지고 잊어버림

> LSTM: 메모장에 중요한 것만 정리하며 필요할 때 다시 꺼내보는 사람 → 정보 관리가 체계적이고 효율적